## Purpose
This is to be used for loading the data loader and loading the model

In [ ]:
# if image_size_for_grid_centers is None:
#     image_size_for_grid_centers = config.data.image_size


In [ ]:
print('')

In [ ]:
# # from disentangle.data_loader.overlapping_dloader import get_overlapping_dset
# from disentangle.data_loader.vanilla_dloader import MultiChDloader
# from disentangle.data_loader.lc_multich_dloader import LCMultiChDloader
# from disentangle.core.data_split_type import DataSplitType
# from disentangle.data_loader.single_channel.single_channel_dloader import SingleChannelDloader
# from disentangle.data_loader.single_channel.single_channel_mc_dloader import SingleChannelMSDloader
# from disentangle.data_loader.pavia2_3ch_dloader import Pavia2ThreeChannelDloader
# from disentangle.data_loader.patch_index_manager import GridAlignement
# from disentangle.data_loader.ht_iba1_ki67_dloader import IBA1Ki67DataLoader
# from disentangle.data_loader.multifile_dset import MultiFileDset

padding_kwargs = {
    'mode':config.data.get('padding_mode','constant'),
}

if padding_kwargs['mode'] == 'constant':
    padding_kwargs['constant_values'] = config.data.get('padding_value',0)

dloader_kwargs = {'overlapping_padding_kwargs':padding_kwargs, 
                  'tiling_mode': tiling_mode,
                  }

In [ ]:
from disentangle.training import create_dataset
train_dset, val_dset = create_dataset(config, data_dir, eval_datasplit_type=eval_datasplit_type,
                                      kwargs_dict=dloader_kwargs)
data_mean, data_std = train_dset.get_mean_std()

In [ ]:
print('')

In [ ]:
!ls /home/ashesh.ashesh/training/disentangle/2301/D3-M10-S0-L3/25

In [ ]:
from disentangle.scripts.run import get_mean_std_dict_for_model
with config.unlocked():
    if old_image_size is not None:
        config.data.image_size = old_image_size

# if config.data.target_separate_normalization is True:
#     mean_fr_model, std_fr_model = train_dset.compute_individual_mean_std()
# else:
#     mean_fr_model, std_fr_model = train_dset.get_mean_std()

# if config.model.model_type == ModelType.LadderVaeSemiSupervised:
#     mean_fr_model = mean_fr_model[None]
#     std_fr_model = std_fr_model[None]

###### Create the input and target mean and std for feeding to the model
mean_dict, std_dict = get_mean_std_dict_for_model(config, train_dset)
# mean_dict = {'input': None, 'target': None}
# std_dict = {'input': None, 'target': None}
# inp_fr_mean, inp_fr_std = train_dset.get_mean_std()
# mean_sq = inp_fr_mean.squeeze()
# std_sq = inp_fr_std.squeeze()
# assert mean_sq[0] == mean_sq[1] and len(mean_sq) == config.data.get('num_channels',2)
# assert std_sq[0] == std_sq[1] and len(std_sq) == config.data.get('num_channels',2)
# mean_dict['input'] = np.mean(inp_fr_mean, axis=1, keepdims=True)
# std_dict['input'] = np.mean(inp_fr_std, axis=1, keepdims=True)

# if config.data.target_separate_normalization is True:
#     target_data_mean, target_data_std = train_dset.compute_individual_mean_std()
# else:
#     target_data_mean, target_data_std = train_dset.get_mean_std()

# mean_dict['target'] = target_data_mean
# std_dict['target'] = target_data_std
###### 
  
model = create_model(config, mean_dict,std_dict)
if os.path.isdir(ckpt_dir):
    ckpt_fpath = get_best_checkpoint(ckpt_dir)
else:
    assert os.path.isfile(ckpt_dir)
    ckpt_fpath = ckpt_dir

print('Loading checkpoint from', ckpt_fpath)
checkpoint = torch.load(ckpt_fpath)

_ = model.load_state_dict(checkpoint['state_dict'], strict=False)
model.eval()
_= model.cuda()

model.set_params_to_same_device_as(torch.Tensor(1).cuda())

print('Loading from epoch', checkpoint['epoch'])

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Model has {count_parameters(model)/1000_000:.3f}M parameters')